In [2]:
import json
import pandas as pd
import numpy as np


file_path = "yelp_academic_dataset_review.json"

line_count = len(open("yelp_academic_dataset_review.json").readlines())



In [6]:



user_ids,business_ids,stars = [],[],[]


count_for_example = 0

with open('yelp_academic_dataset_review.json') as json_file:
     for line in json_file:
        
            statement = json.loads(line)
            print(statement)
            user_ids += [statement["user_id"]]
            print(user_ids)
            business_ids += [statement["business_id"]]
            print(business_ids)
            stars += [statement["stars"]]
            print(stars)
            
           
            
ratings = pd.DataFrame(
{"user_id" : user_ids, "business_id" : business_ids, "rating" : stars}
)




337637

In [7]:

#ratings 모델을 matrix 형태로 변환

ratings_training = training_set.pivot_table(
    index = 'user_id',
    columns = 'business_id',
    values = 'rating',
    aggfunc = 'median'        #같은 유저가 같은 business에 대한 평가를 할 때 median으로 사용
).fillna(0)

test_set = training_set.pivot_table(
    index = 'user_id',
    columns = 'business_id',
    values = 'rating',
    aggfunc = 'median'        #같은 유저가 같은 business에 대한 평가를 할 때 median으로 사용
).fillna(0)

IndexError: index 132374448 is out of bounds for axis 0 with size 132364585

In [92]:
ratings = ratings_training.to_numpy()

print(ratings)

print(ratings.shape)

[[4. 0.]
 [5. 0.]
 [4. 0.]
 [0. 2.]
 [0. 4.]
 [5. 0.]
 [0. 1.]
 [0. 5.]
 [4. 0.]
 [4. 0.]
 [4. 0.]
 [0. 4.]
 [4. 0.]]
(13, 2)


In [93]:
class MatrixFactorization():
    
    def __init__(self, rating,latent, learning_rate, weight, epochs):
        
        self.rating = rating
        self.latent = latent
        self.learning_rate = learning_rate
        self.weight = weight
        self.num_user, self.num_items = rating.shape
        self.epochs = epochs
        
    def fit(self):
        
        self.users_latent = np.random.normal(size=(self.num_users, self.latent))   #latent matrix 생성
        self.items_latent = np.random.normal(size=(self.num_items, self.latent))
        
        self.users_bias = np.zeros(self.num_users)                                #bias 0으로 생성
        self.items_bias = np.zeros(self.num_items)
        self.bias = np.mean(self.rating[np.where(self.rating != 0)])
        
        
        
        for epoch in range(self.epochs):                                    # 총 epochs만큼 반복해서 SGD 이행
            
            for i in range(self.num_items):
                for j in range(self.num_itmes):
                    if self.rating[i,j] > 0:
                        self.gradient_descent(i,j, self.rating)
            cost = self.cost()
            self.training.append((epoch, cost))
            
    def cost(self):
        
        x_index,y_index = self.rating.nonzero()
        prediction = self.calculated_matrix()
        cost = 0
        
        for i,j in zip(x_index,y_index):
            cost += pow(rating[i,j] - prediction[i,j],2)
            
        rmse_value = np.sqrt(cost/len(x_index))
        
        return rmse_value
    
    
    def calculated_matrix(self):
        
        return self.bias + self.users_bias[:,np.newaxis] + self.items_bias[np.newaxis, : ] + self.users_latent.dot(self.items_latent.T)
        
        
        
        
        
    def gradient(self, error, user_index, item_index):
        
        d_users = (error * self.items_latent[item_index,:]) - (self.weight * self.users_latent[user_index, :])
        d_items = (error * self.users_latent[user_index, :]) - (self.weight * self.items_latent[item_index, :])
        
        return d_users,d_items
    
    def gradient_descent(self,user_index, item_index, rating):
        
        prediction = self.predicted(user_index,item_index)
        error = rating - prediction
        
        self.users_bias[user_index] -= self.learning_rate * (error - self.weight*self.users_bias[user_index])
        self.items_bias[item_index] -= self.learning_rate * (error - self.weight*self.items_bias[item_index])
        
        d_users,d_items = self.gradient(error, user_index,item_index)
        
        self.users_latent[user_index, : ] -= self.learning_rate * d_users
        self.items_latent[item_index, : ] -= self.learning_rate * d_items
        
        
        
    def predicted(self,user_index,item_index):
        
        prediction_values = self.bias + self.users_bias[user_index]+self.items_bias[item_index] + self.users_latent[user_index,:].dot(self.items_latent[item_index, : ].T)
        
        return prediction_values
    

    
    
    

        
        

In [ ]:
model = MatrixFacotrization(ratings,latent=5, learning_rate = 0.05, weight = 0.05, epochs = 100)

model.fit() 

model.calculated_matrix()

